## Data Cleaning and Renaming all Files

In [15]:
import os
import pandas as pd
import re
import emoji

input_folder = r'D:\SEM_2\MFC\graphAssignment\LinkedIn Data Public'
output_folder = r'D:\SEM_2\MFC\graphAssignment\cleanedFiles'

os.makedirs(output_folder, exist_ok=True)

cleaned_files = []
skipped_files = []
existing_names = set()

print("🔧 Setup complete. Input and output folders are ready.")


🔧 Setup complete. Input and output folders are ready.


In [16]:
def is_binary(file_path):
    try:
        with open(file_path, 'rb') as f:
            chunk = f.read(1024)
            return b'\x00' in chunk
    except Exception:
        return True

def clean_text(text):
    if isinstance(text, str):
        text = emoji.replace_emoji(text, "")
        text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
        text = text.title().strip()
    return text

def parse_fallback(file_path):
    try:
        with open(file_path, 'r', encoding='ISO-8859-1', errors='ignore') as f:
            lines = f.readlines()

        cleaned_rows = []
        for line in lines:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            if ',' in line:
                cleaned_rows.append(line.split(','))
            elif '\t' in line:
                cleaned_rows.append(line.split('\t'))
            elif '|' in line:
                cleaned_rows.append(line.split('|'))

        if not cleaned_rows:
            return None

        max_len = max(len(row) for row in cleaned_rows)
        cleaned_rows = [row + [''] * (max_len - len(row)) for row in cleaned_rows]
        df = pd.DataFrame(cleaned_rows[1:], columns=cleaned_rows[0])
        return df

    except Exception as e:
        print(f"[FALLBACK ERROR] Failed to parse manually: {e}")
        return None


In [17]:
def clean_csv(file_path, output_path):
    print(f"\n📂 Processing file: {os.path.basename(file_path)}")

    try:
        encodings = ['utf-8', 'ISO-8859-1', 'utf-16']
        df = None
        used_encoding = None

        for enc in encodings:
            try:
                df = pd.read_csv(file_path, encoding=enc, on_bad_lines='skip')
                used_encoding = enc
                break
            except Exception:
                continue

        if df is None or df.empty:
            print(f'⚠️ Fallback triggered for: {os.path.basename(file_path)}')
            df = parse_fallback(file_path)
            used_encoding = 'fallback'

        if df is None or df.empty:
            print(f"❌ Could not recover or clean: {os.path.basename(file_path)}")
            skipped_files.append(os.path.basename(file_path))
            return

        print(f"✅ File read successfully with encoding: {used_encoding}")
        print(f"🔎 Original shape: {df.shape}")

        df = df.drop_duplicates()
        df.columns = [clean_text(col) for col in df.columns]

        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].apply(clean_text)

        df = df.dropna(how='all')
        print(f"📉 After cleaning shape: {df.shape}")
        print(f"📑 Cleaned columns: {list(df.columns)}")

        if len(df.columns) > 3:
            if all(col in df.columns for col in ['First Name', 'Last Name', 'Company']):
                df = df[['First Name', 'Last Name', 'Company']]
                print("📌 Selected 3 main columns.")
        elif len(df.columns) == 3:
            df.columns = ['First Name', 'Last Name', 'Company']
        elif len(df.columns) == 1:
            new_cols = df.iloc[:, 0].str.split(expand=True, n=2)
            new_cols.columns = ['First Name', 'Last Name', 'Company'][:new_cols.shape[1]]
            df = new_cols
            print("🧩 Split single column into multiple.")

        for col in df.columns:
            long_values = df[col].apply(lambda x: len(str(x)) > 100)
            if long_values.any():
                print(f"✂️ Trimming long text in column: {col}")
                df = df[~long_values]

        display(df.head())
        df.to_csv(output_path, index=False)
        print(f"💾 Cleaned data saved to: {output_path}")
        cleaned_files.append(os.path.basename(file_path))

    except Exception as e:
        print(f'[ERROR] Could not clean {os.path.basename(file_path)}: {e}')
        skipped_files.append(os.path.basename(file_path))


In [18]:
for file_name in os.listdir(input_folder):
    original_path = os.path.join(input_folder, file_name)

    if not os.path.isfile(original_path):
        continue

    ext = os.path.splitext(file_name)[1].lower()
    base_name = os.path.splitext(file_name)[0]
    cleaned_file_name = base_name + '.csv'
    cleaned_path = os.path.join(output_folder, cleaned_file_name)

    if ext == '.xlsx':
        try:
            df = pd.read_excel(original_path)
            df.to_csv(cleaned_path, index=False)
            print(f"[CONVERTED] {file_name} -> {cleaned_file_name}")
        except Exception as e:
            print(f"[ERROR] Failed to convert {file_name}: {e}")
            skipped_files.append(file_name)
            continue

    input_path_for_cleaning = cleaned_path if ext == '.xlsx' else original_path
    clean_csv(input_path_for_cleaning, cleaned_path)



📂 Processing file: Aaditya_Raj - Aaditya Raj.csv
✅ File read successfully with encoding: utf-8
🔎 Original shape: (267, 3)
📉 After cleaning shape: (266, 3)
📑 Cleaned columns: ['First Name', 'Last Name', 'Company']


,First Name,Last Name,Company
0,Harsh Kumar,Singh,Avenues Iit Bombay
1,Saurabh,Singh,NaN
2,Anamika,Kumari,NaN
3,Hariom,Parmar,Anudip Foundation
4,Sukhe,,Minerva Academy Football Club


💾 Cleaned data saved to: D:\SEM_2\MFC\graphAssignment\cleanedFiles\Aaditya_Raj - Aaditya Raj.csv

📂 Processing file: Abhishek_Singh - Abhishek Singh.csv
✅ File read successfully with encoding: utf-8
🔎 Original shape: (164, 3)
📉 After cleaning shape: (161, 3)
📑 Cleaned columns: ['First Name', 'Last Name', 'Company']


,First Name,Last Name,Company
0,Tarang,Pandey,Shiksha Bank
1,Aarti,Patil,NaN
2,Lokesh,Prajapat Hr,Jawahar Navodaya Vidyalaya Jnv
3,Prabhat,Kumar,NaN
4,Baikuntha,Nath Mangual,NaN


💾 Cleaned data saved to: D:\SEM_2\MFC\graphAssignment\cleanedFiles\Abhishek_Singh - Abhishek Singh.csv

📂 Processing file: Aditya_Singh - Aditya NO-LASTNAME.csv
✅ File read successfully with encoding: utf-8
🔎 Original shape: (291, 5)
📉 After cleaning shape: (289, 5)
📑 Cleaned columns: ['First Name', 'Last Name', 'Unnamed 2', 'Unnamed 3', 'Company']
📌 Selected 3 main columns.


...

💾 Cleaned data saved to: D:\SEM_2\MFC\graphAssignment\cleanedFiles\Vivek_Singh - Vivek Singh.csv

📂 Processing file: YuvrajSingh_Bhati - Yuvraj Bhati.csv
✅ File read successfully with encoding: utf-8
🔎 Original shape: (175, 3)
📉 After cleaning shape: (175, 3)
📑 Cleaned columns: ['First Name', 'Last Name', 'Company']


,First Name,Last Name,Company
0,Ajay,Kumar,Debate Club
1,Rahul,Kumar,NaN
2,Pushpraj,Singh,NaN
3,Sandeep,Kumar,Google Developer Experts
4,Aditya,Singh,NaN


💾 Cleaned data saved to: D:\SEM_2\MFC\graphAssignment\cleanedFiles\YuvrajSingh_Bhati - Yuvraj Bhati.csv
[CONVERTED] Yuvraj_Chirag - Yuvraj Chirag.xlsx -> Yuvraj_Chirag - Yuvraj Chirag.csv

📂 Processing file: Yuvraj_Chirag - Yuvraj Chirag.csv
✅ File read successfully with encoding: utf-8
🔎 Original shape: (763, 3)
📉 After cleaning shape: (753, 3)
📑 Cleaned columns: ['First Name', 'Last Name', 'Company']


,First Name,Last Name,Company
0,Sachin,Yadav,NaN
1,Punam,Mandal,NaN
2,Aman,Singh Chauhan,NaN
3,Mani,Kumar,NaN
4,Amar,Kumar,Yoho


💾 Cleaned data saved to: D:\SEM_2\MFC\graphAssignment\cleanedFiles\Yuvraj_Chirag - Yuvraj Chirag.csv


In [19]:
for file_name in os.listdir(output_folder):
    if not file_name.endswith('.csv'):
        continue

    old_path = os.path.join(output_folder, file_name)

    if '-' in file_name:
        new_name = file_name.split('-', 1)[1].strip()
    else:
        new_name = file_name

    new_name = os.path.splitext(new_name)[0].title().strip() + '.csv'

    if new_name in existing_names:
        print(f"[DUPLICATE] Skipping duplicate file: {new_name}")
        os.remove(old_path)
        continue

    new_path = os.path.join(output_folder, new_name)
    os.rename(old_path, new_path)
    print(f"[RENAMED] {file_name} -> {new_name}")
    existing_names.add(new_name)


[RENAMED] Aaditya_Raj - Aaditya Raj.csv -> Aaditya Raj.csv
[RENAMED] Abhishek_Singh - Abhishek Singh.csv -> Abhishek Singh.csv
[RENAMED] Aditya_Singh - Aditya NO-LASTNAME.csv -> Aditya No-Lastname.csv
[RENAMED] Afzal_Raza - Afzl Raza.csv -> Afzl Raza.csv
[RENAMED] Ajay Jatav Connections-1 - Ajay Jatav.csv -> 1 - Ajay Jatav.csv
[RENAMED] Ajit_Yadav - Ajit Yadav.csv -> Ajit Yadav.csv
[RENAMED] Akanksha_Kushwaha - Akanksha.csv -> Akanksha.csv
[RENAMED] Alok_raj - Alok Raj.csv -> Alok Raj.csv
[RENAMED] Aman_ Adarsh.csv -> Aman_ Adarsh.csv
[RENAMED] Aman_Singh - Aman Singh.csv -> Aman Singh.csv
[RENAMED] amit_kumar - Amit Kumar.csv -> Amit Kumar.csv
[RENAMED] Anamika_Kumari - Anamika Kumari.csv -> Anamika Kumari.csv
[RENAMED] Anand_Pandey - Anand Pandey.csv -> Anand Pandey.csv
[RENAMED] Anoop_Kumar - ANOOP KUMAR.csv -> Anoop Kumar.csv
[RENAMED] Anshu_Kumar - Anshu Kumar.csv -> Anshu Kumar.csv
[RENAMED] Anuradha_Tiwari - Anuradha Tiwari.csv -> Anuradha Tiwari.csv
[RENAMED] Anushri_Mishra - A

[RENAMED] Janu_Chaudhary - Janu Chaudhary.csv -> Janu Chaudhary.csv
[RENAMED] KARANPAL_SINGH_RANAWAT - KARANPAL SINGH RANAWAT.csv -> Karanpal Singh Ranawat.csv
[RENAMED] khushi_narwariya - Khushi Narwariya.csv -> Khushi Narwariya.csv
[RENAMED] Kuldeep_Saraswat - Kuldeep saraswat.csv -> Kuldeep Saraswat.csv
[RENAMED] Lakhan_Rathore - Lakhan Rathore.csv -> Lakhan Rathore.csv
[RENAMED] linkedin list - Nidhi Kumari.csv -> Nidhi Kumari.csv
[RENAMED] linkedin_Connections - SNEHA SHAW.csv -> Sneha Shaw.csv
[RENAMED] Linked_in_connection - Samina Sultana.csv -> Samina Sultana.csv
[RENAMED] Maneesh_Sakhwar - Maneesh Sakhwar.csv -> Maneesh Sakhwar.csv
[RENAMED] Manish_Tiwari - MANISH KUMAR TIWARI.csv -> Manish Kumar Tiwari.csv
[RENAMED] Manoj K. Connections - MANOJ KHARKAR.csv -> Manoj Kharkar.csv
[RENAMED] Manoj_Dewda - Manoj Dewda.csv -> Manoj Dewda.csv
[RENAMED] Mausam_kumari - Mausam kumari.csv -> Mausam Kumari.csv
[RENAMED] Mayank_Raj - Mayank Raj.csv -> Mayank Raj.csv
[RENAMED] Mehtab_Alam

In [20]:
print("\n--- ✅ Summary ---")
print("Cleaned files:")
for f in cleaned_files:
    print(f" - {f}")

print("\nSkipped or uncleaned files:")
for f in skipped_files:
    print(f" - {f}")



--- ✅ Summary ---
Cleaned files:
 - Aaditya_Raj - Aaditya Raj.csv
 - Abhishek_Singh - Abhishek Singh.csv
 - Aditya_Singh - Aditya NO-LASTNAME.csv
 - Afzal_Raza - Afzl Raza.csv
 - Ajay Jatav Connections-1 - Ajay Jatav.csv
 - Ajit_Yadav - Ajit Yadav.csv
 - Akanksha_Kushwaha - Akanksha.csv
 - Alok_raj - Alok Raj.csv
 - Aman_ Adarsh.csv
 - Aman_Singh - Aman Singh.csv
 - amit_kumar - Amit Kumar.csv
 - Anamika_Kumari - Anamika Kumari.csv
 - Anand_Pandey - Anand Pandey.csv
 - Anoop_Kumar - ANOOP KUMAR.csv
 - Anshu_Kumar - Anshu Kumar.csv
 - Anuradha_Tiwari - Anuradha Tiwari.csv
 - Anushri_Mishra - Anushri Mishra.csv
 - Aradhya_Patel - Aradhya Patel.csv
 - Arjun Kadam - Arjun Kadam.csv
 - Arpita_Tripathi - Arpita Tripathi.csv
 - Arun_Singh - ARUN KUMAR.csv
 - aryan_saini - Aryan Saini.csv
 - Ashwin_Yadav - Ashwin Yadav.csv
 - Ayush_Kumar - Ayush Kumar.csv
 - Ayush_yadav - AYUSH YADAV.csv
 - Bhagwati_Chouhan - Bhagwati NO-LASTNAME.csv
 - Bhaskar_mahato - Bhaskar Mahato.csv
 - ByagariPraveen_Ku

### Some Manual Data Cleaning 

#### 1. Renaming "1 - Ajay Jatav.csv" and "1 - Aman Adarsh.csv" to "Ajay Jatav.csv" and "Aman Adarsh.csv" respectively.

#### 2. Samina_Sultana.csv, Aman_Adrash and Sneha_Shaw removed because they have duplicates.

#### 3.Anamika kumari.csv have duplicate row of column name's so that I removed.

#### 4.Shlok Gupta.csv and Ompal Yadav.csv, both files have 5 columns and didn't have named them as ["First Name", "Last Name","Company"], So shlok Gupta.csv have extra row at the top so I remove that and Ompal Yadav.csv have also extra row and some unwanted column space between them so removed that also.

#### 5. At the Start I also renamed "Vivek Singh- Vivek Kumar.csv" to "Vivek Singh- Vivek Singh.csv" because "Vivek Kumar- Vivek Kumar.csv" already exist in raw file folder and I choose Names (for rename) from right side of underscore.

#### 6. I clean pooran singh.csv manually like giving space and many things.

In [24]:
import os

# Specify the folder path
folder_path = r'D:\SEM_2\MFC\graphAssignment\cleanedFiles'

# Count files (not directories) in the folder
file_count = sum(1 for entry in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, entry)))

print(f"📁 Total number of files in '{folder_path}': {file_count}")


📁 Total number of files in 'D:\SEM_2\MFC\graphAssignment\cleanedFiles': 126
